In [40]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [ ]:
#enter the estimates page you would like to scrape into url, requests and bs4 pull the data 

url = "https://www.ontario.ca/page/expenditure-estimates-ministry-health-2024-25"
page = requests.get(url)
soup = BeautifulSoup(page.text,'html')

In [5]:
#extract the tables from the webpage
tables = soup.find_all('table')

In [ ]:
#create table1 dataframe
header = tables[0].find_all('th')
rows = tables[0].find_all('tr')
column_headers = [title.text.strip() for title in header]
table1 = pd.DataFrame(columns = column_headers)
caption = tables[0].find('caption')
table_name = caption.text.strip()
clean_text = re.sub(r"\s", "_", table_name )




In [ ]:
#grab row data and put it in that pandas dataframe
for row in rows[1:]:
    row_data = row.find_all('td')
    individual_row_data = [data.text.strip() for data in row_data]
    length = len(table1)
    table1.loc[length] = individual_row_data

#assign
#globals()[clean_text] = table1

In [ ]:
headerdict = {}  
tables_dict = {} 

for i in range(len(tables)):
    headers = tables[i].find_all('th')  
    column_names = [a.text.strip() for a in headers] 
    headerdict[i] = column_names  
    
    # Create an empty DataFrame with the column names
    tables_dict[i] = pd.DataFrame(columns=column_names)
    
    rows = tables[i].find_all('tr')  # Find all <tr> tags (rows)
    for row in rows[1:]:  # Skip the first row which contains the headers
        row_data = row.find_all('td')  # Extract all <td> tags (columns)
        individual_row_data = [data.text.strip() for data in row_data]  # Clean up the text
        
        # Check if the number of columns matches
        if len(individual_row_data) == len(column_names):
            # Append the row to the DataFrame
            length = len(tables_dict[i])  # Get the current length of the DataFrame
            tables_dict[i].loc[length] = individual_row_data  # Add the row
        elif len(individual_row_data) < len(column_names):
            # Pad the row with None or NaN if there are missing columns
            while len(individual_row_data) < len(column_names):
                individual_row_data.append(None)  # Add None to match the number of columns
            length = len(tables_dict[i])
            tables_dict[i].loc[length] = individual_row_data  # Add the padded row
        else:
            # If there are extra columns, we can choose to skip the row or trim it
            print(f"Skipping row {row} as it has more columns than expected.")

# Check the result
print(headerdict)
print(tables_dict)



{0: ['Vote', 'Program', 'Estimates2024-25 $', 'Estimates2023-24 $', 'DifferenceBetween2024-25and2023-24 $', 'Actual2022-23 $'], 1: ['Vote', 'Program', 'Estimates2024-25 $', 'Estimates2023-24 $', 'DifferenceBetween2024-25and2023-24 $', 'Actual2022-23 $'], 2: ['Vote', 'Program', 'Estimates2024-25 $', 'Estimates2023-24 $', 'DifferenceBetween2024-25and2023-24 $', 'Actual2022-23 $'], 3: ['Vote', 'Program', 'Estimates2024-25 $', 'Estimates2023-24 $', 'DifferenceBetween2024-25and2023-24 $', 'Actual2022-23 $'], 4: ['Operating expense', 'Estimates2023-24 $', 'Actual2022-23 $', 'Total operating expense previously published\xa0footnote 1[1]', 'Government reorganization', 'Transfer of functions to other Ministries', 'Restated total operating expense'], 5: ['Item number', 'Item', 'Estimates2024-25 $', 'Estimates2023-24 $', 'DifferenceBetween2024-25and2023-24 $', 'Actual2022-23 $'], 6: ['Item number', 'Item', 'Estimates2024-25 $', 'Estimates2023-24 $', 'DifferenceBetween2024-25and2023-24 $', 'Actual

In [92]:
output_file = 'tables_data.xlsx'

# Use pandas ExcelWriter to write to an Excel file
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    # Loop through the dictionary and write each DataFrame to a separate sheet
    for i, df in tables_dict.items():
        sheet_name = f'Table_{i}'  # Define the sheet name (e.g., 'Table_0', 'Table_1', etc.)
        df.to_excel(writer, sheet_name=sheet_name, index=False)  # Write DataFrame to sheet, without the index column


In [93]:
import os

current_directory = os.getcwd()
print("Current Working Directory:", current_directory)

Current Working Directory: c:\Users\mt886\OneDrive\Documents\GitHub\Estimates-Web-Scraper
